In [ ]:
# -*- coding: utf-8 -*-
"""
PRÁCTICA COMPLETA - MODELOS SUPERVISADO Y NO SUPERVISADO
Análisis Predictivo del Rendimiento Académico
"""

# ============================================================================
# 1. IMPORTACIÓN DE LIBRERÍAS
# ============================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
print("✅ Librerías importadas correctamente")

# ============================================================================
# 2. CARGA Y EXPLORACIÓN DE DATOS
# ============================================================================
print("=" * 70)
print("2. CARGA Y EXPLORACIÓN DE DATOS")
print("=" * 70)

try:
    # Cargar dataset
    df = pd.read_csv('academic_performance_master.csv')
    print(f"✅ Dataset cargado exitosamente")
    print(f"   • Dimensión: {df.shape[0]} filas x {df.shape[1]} columnas")
    print(f"   • Columnas: {list(df.columns)}")
    
except FileNotFoundError:
    print("❌ ERROR: No se encontró 'academic_performance_master.csv'")
    print("📝 Creando dataset de ejemplo para continuar...")
    
    # Crear datos de ejemplo
    np.random.seed(42)
    n_estudiantes = 150
    
    datos = {
        'Estudiante_ID': [f'EST{i:03d}' for i in range(n_estudiantes)],
        'Nombre': [f'Estudiante_{i}' for i in range(n_estudiantes)],
        'Edad': np.random.randint(18, 30, n_estudiantes),
        'Genero': np.random.choice(['M', 'F'], n_estudiantes),
        'Carrera': np.random.choice(['Ingeniería', 'Medicina', 'Derecho', 'Administración'], n_estudiantes),
        'Semestre': np.random.randint(1, 10, n_estudiantes),
        'Asistencia': np.random.normal(85, 10, n_estudiantes).clip(60, 100).astype(int),
        'Tareas_entregadas': np.random.randint(5, 20, n_estudiantes),
        'Participacion_clase': np.random.normal(7, 2, n_estudiantes).clip(0, 10).astype(int),
        'Horas_estudio': np.random.normal(12, 4, n_estudiantes).clip(2, 25).astype(int),
        'Nota_parcial1': np.random.normal(75, 15, n_estudiantes).clip(30, 100).astype(int),
        'Nota_parcial2': np.random.normal(72, 18, n_estudiantes).clip(30, 100).astype(int),
        'Nota_final': np.random.normal(70, 20, n_estudiantes).clip(0, 100).astype(int),
        'Nivel': np.random.choice(['Licenciatura', 'Maestría'], n_estudiantes, p=[0.8, 0.2])
    }
    
    df = pd.DataFrame(datos)
    df.to_csv('academic_performance_master.csv', index=False)
    print("✅ Dataset de ejemplo creado y guardado")

# Mostrar información básica
print(f"\n📊 INFORMACIÓN DEL DATASET:")
print(df.info())

print(f"\n📈 ESTADÍSTICAS DESCRIPTIVAS:")
print(df.describe())

print(f"\n🔍 VALORES NULOS POR COLUMNA:")
print(df.isnull().sum())

print(f"\n🔄 VALORES DUPLICADOS: {df.duplicated().sum()}")

print(f"\n📋 DISTRIBUCIÓN DE VARIABLES CATEGÓRICAS:")
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col != 'Estudiante_ID':
        print(f"\n{col}:")
        print(df[col].value_counts().head())

# ============================================================================
# 3. PREPARACIÓN DEL DATASET
# ============================================================================
print("\n" + "=" * 70)
print("3. PREPARACIÓN DEL DATASET")
print("=" * 70)

df_clean = df.copy()

# 3.1 Manejo de valores nulos
print("🔧 MANEJO DE VALORES NULOS:")
for col in df_clean.columns:
    null_count = df_clean[col].isnull().sum()
    if null_count > 0:
        if df_clean[col].dtype == 'object':
            df_clean[col].fillna('Desconocido', inplace=True)
            print(f"   • {col}: {null_count} nulos → 'Desconocido'")
        else:
            median_val = df_clean[col].median()
            df_clean[col].fillna(median_val, inplace=True)
            print(f"   • {col}: {null_count} nulos → {median_val:.2f}")

# 3.2 Eliminar duplicados
if df_clean.duplicated().sum() > 0:
    df_clean.drop_duplicates(inplace=True)
    print(f"\n🗑️  Eliminados {df.duplicated().sum()} registros duplicados")

# 3.3 Crear variable objetivo
print("\n🎯 CREANDO VARIABLE OBJETIVO:")
df_clean['Aprobado'] = (df_clean['Nota_final'] >= 70).astype(int)
print(f"   • Aprobados (1): {df_clean['Aprobado'].sum()} estudiantes")
print(f"   • Reprobados (0): {len(df_clean) - df_clean['Aprobado'].sum()} estudiantes")
print(f"   • Tasa de aprobación: {df_clean['Aprobado'].mean()*100:.1f}%")

# 3.4 Preparar datos numéricos para modelo
print("\n📊 PREPARANDO DATOS PARA MODELOS:")

# Seleccionar columnas numéricas relevantes
numeric_features = ['Asistencia', 'Tareas_entregadas', 'Participacion_clase', 
                    'Horas_estudio', 'Nota_parcial1', 'Nota_parcial2']

# Verificar qué columnas existen realmente
existing_features = [col for col in numeric_features if col in df_clean.columns]
print(f"   • Características numéricas encontradas: {existing_features}")

# Si no hay suficientes características, usar todas las numéricas
if len(existing_features) < 3:
    existing_features = df_clean.select_dtypes(include=[np.number]).columns.tolist()
    # Excluir columnas no relevantes
    exclude = ['Aprobado', 'Nota_final']
    existing_features = [col for col in existing_features if col not in exclude]
    print(f"   • Usando todas las numéricas: {existing_features}")

X = df_clean[existing_features]
y = df_clean['Aprobado']

# 3.5 Estandarizar características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 3.6 Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\n📈 DIVISIÓN DE DATOS:")
print(f"   • Conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"   • Conjunto de prueba: {X_test.shape[0]} muestras")
print(f"   • Características: {X_train.shape[1]} variables")

# ============================================================================
# 4. MODELO SUPERVISADO - CLASIFICACIÓN
# ============================================================================
print("\n" + "=" * 70)
print("4. MODELO SUPERVISADO - REGRESIÓN LOGÍSTICA")
print("=" * 70)

# 4.1 Entrenar modelo
print("🚀 ENTRENANDO MODELO DE REGRESIÓN LOGÍSTICA...")
model_lr = LogisticRegression(random_state=42, max_iter=1000)
model_lr.fit(X_train, y_train)

# 4.2 Predicciones
y_pred = model_lr.predict(X_test)
y_pred_proba = model_lr.predict_proba(X_test)[:, 1]

# 4.3 Métricas
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"✅ MODELO ENTRENADO EXITOSAMENTE")
print(f"\n📊 MÉTRICAS DEL MODELO:")
print(f"   • Accuracy: {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f"   • Precisión: {precision_score(y_test, y_pred):.4f}")
print(f"   • Recall: {recall_score(y_test, y_pred):.4f}")
print(f"   • F1-Score: {f1_score(y_test, y_pred):.4f}")

print(f"\n📋 MATRIZ DE CONFUSIÓN:")
print(conf_matrix)

print(f"\n📄 REPORTE DE CLASIFICACIÓN:")
print(class_report)

# 4.4 Importancia de características
if hasattr(model_lr, 'coef_'):
    importance = pd.DataFrame({
        'Variable': existing_features,
        'Importancia': np.abs(model_lr.coef_[0])
    }).sort_values('Importancia', ascending=False)
    
    print(f"\n🔝 IMPORTANCIA DE VARIABLES:")
    print(importance.to_string(index=False))

# 4.5 Visualización
print("\n🎨 GENERANDO VISUALIZACIONES...")

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Matriz de confusión
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Reprobado', 'Aprobado'],
            yticklabels=['Reprobado', 'Aprobado'], 
            ax=axes[0])
axes[0].set_title('Matriz de Confusión')
axes[0].set_xlabel('Predicción')
axes[0].set_ylabel('Real')

# Importancia de variables
importance_sorted = importance.sort_values('Importancia')
axes[1].barh(importance_sorted['Variable'], importance_sorted['Importancia'])
axes[1].set_title('Importancia de Variables')
axes[1].set_xlabel('Importancia Absoluta')
axes[1].set_ylabel('Variable')

# Distribución de probabilidades
axes[2].hist(y_pred_proba[y_test == 0], alpha=0.5, label='Reprobados', bins=20, color='red')
axes[2].hist(y_pred_proba[y_test == 1], alpha=0.5, label='Aprobados', bins=20, color='green')
axes[2].set_title('Distribución de Probabilidades Predichas')
axes[2].set_xlabel('Probabilidad de Aprobar')
axes[2].set_ylabel('Frecuencia')
axes[2].legend()
axes[2].axvline(x=0.5, color='black', linestyle='--', linewidth=1)

plt.tight_layout()
plt.savefig('metricas_supervisado.png', dpi=100, bbox_inches='tight')
plt.show()
print("✅ Visualización guardada como 'metricas_supervisado.png'")

# ============================================================================
# 5. MODELO NO SUPERVISADO - CLUSTERING
# ============================================================================
print("\n" + "=" * 70)
print("5. MODELO NO SUPERVISADO - K-MEANS CLUSTERING")
print("=" * 70)

# 5.1 Seleccionar características para clustering
print("🔍 PREPARANDO DATOS PARA CLUSTERING...")
clustering_features = ['Asistencia', 'Nota_final', 'Tareas_entregadas']
# Verificar qué características están disponibles
available_features = [f for f in clustering_features if f in df_clean.columns]

if len(available_features) < 2:
    # Si no hay suficientes, usar las primeras 3 numéricas
    available_features = df_clean.select_dtypes(include=[np.number]).columns.tolist()[:3]
    print(f"⚠️  Usando características alternativas: {available_features}")

X_cluster = df_clean[available_features].copy()

# 5.2 Estandarizar
scaler_cluster = StandardScaler()
X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)

# 5.3 Método del codo
print("📈 APLICANDO MÉTODO DEL CODO...")
inertias = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_cluster_scaled)
    inertias.append(kmeans.inertia_)

# Determinar k óptimo (simple)
inertia_diffs = np.diff(inertias)
k_optimal = np.argmax(inertia_diffs) + 2  # +2 porque empezamos en k=1

print(f"   • K óptimo sugerido: {k_optimal}")

# 5.4 Aplicar K-means
print(f"🎯 APLICANDO K-MEANS CON K={k_optimal}...")
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_cluster_scaled)

df_clean['Cluster'] = clusters

print(f"\n📊 DISTRIBUCIÓN DE CLUSTERS:")
print(df_clean['Cluster'].value_counts().sort_index())

print(f"\n📈 ESTADÍSTICAS POR CLUSTER:")
cluster_stats = df_clean.groupby('Cluster').agg({
    'Nota_final': ['mean', 'std', 'min', 'max'],
    'Asistencia': ['mean', 'std'],
    'Tareas_entregadas': ['mean', 'std'],
    'Aprobado': 'mean'
})

print(cluster_stats.round(2))

# 5.5 Interpretación
print(f"\n👥 INTERPRETACIÓN DE CLUSTERS:")
for i in range(k_optimal):
    cluster_data = df_clean[df_clean['Cluster'] == i]
    print(f"\n   CLUSTER {i} (n={len(cluster_data)}):")
    print(f"     • Nota final: {cluster_data['Nota_final'].mean():.1f}")
    print(f"     • Asistencia: {cluster_data['Asistencia'].mean():.1f}%")
    print(f"     • Tareas: {cluster_data['Tareas_entregadas'].mean():.1f}")
    print(f"     • Aprobación: {cluster_data['Aprobado'].mean()*100:.1f}%")

# 5.6 Visualización
print("\n🎨 GENERANDO VISUALIZACIONES DE CLUSTERING...")

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Método del codo
axes[0].plot(k_range, inertias, 'bo-')
axes[0].axvline(x=k_optimal, color='r', linestyle='--', label=f'K óptimo = {k_optimal}')
axes[0].set_xlabel('Número de Clusters (k)')
axes[0].set_ylabel('Inercia')
axes[0].set_title('Método del Codo')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Gráfico de clusters
if len(available_features) >= 2:
    scatter = axes[1].scatter(df_clean[available_features[0]], 
                             df_clean[available_features[1]], 
                             c=df_clean['Cluster'], cmap='viridis', alpha=0.6, s=50)
    axes[1].set_xlabel(available_features[0])
    axes[1].set_ylabel(available_features[1])
    axes[1].set_title(f'Clustering: {available_features[0]} vs {available_features[1]}')
    plt.colorbar(scatter, ax=axes[1], label='Cluster')
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('clustering_resultados.png', dpi=100, bbox_inches='tight')
plt.show()
print("✅ Visualización guardada como 'clustering_resultados.png'")

# ============================================================================
# 6. ANÁLISIS COMPARATIVO Y CONCLUSIONES
# ============================================================================
print("\n" + "=" * 70)
print("6. ANÁLISIS COMPARATIVO Y CONCLUSIONES")
print("=" * 70)

print("\n📊 COMPARACIÓN DE MODELOS:")
print("-" * 40)
print("🔮 MODELO SUPERVISADO (Regresión Logística):")
print(f"   • Tipo: Clasificación binaria")
print(f"   • Precisión: {accuracy:.2%}")
print(f"   • Variables importantes: {importance.head(3)['Variable'].tolist()}")
print(f"   • Fortalezas: Buen poder predictivo, interpretable")
print(f"   • Limitaciones: Asume relación lineal")

print(f"\n🔍 MODELO NO SUPERVISADO (K-means):")
print(f"   • Tipo: Clustering")
print(f"   • Clusters identificados: {k_optimal}")
print(f"   • Patrones encontrados: Grupos con comportamientos similares")
print(f"   • Fortalezas: Descubre patrones ocultos, no requiere etiquetas")
print(f"   • Limitaciones: Requiere definir k, sensible a outliers")

print(f"\n🤔 ¿QUÉ MODELO ES MEJOR?")
print("-" * 40)
print(f"   Depende del objetivo:")
print(f"   1. Para PREDECIR aprobación: Modelo Supervisado")
print(f"      • Proporciona probabilidades específicas")
print(f"      • Alta precisión para identificar estudiantes en riesgo")
print(f"      • Útil para intervenciones tempranas")

print(f"\n   2. Para ENTENDER patrones: Modelo No Supervisado")
print(f"      • Identifica perfiles de estudiantes")
print(f"      • Útil para estrategias pedagógicas diferenciadas")
print(f"      • Ayuda en la segmentación para tutorías")

print(f"\n🚀 RECOMENDACIÓN:")
print(f"   Usar ambos modelos complementariamente:")
print(f"   - K-means para segmentar a los estudiantes en grupos")
print(f"   - Regresión logística para predecir riesgo dentro de cada grupo")
print(f"   - Esto permite intervenciones personalizadas más efectivas")

# Guardar datos procesados
df_clean.to_csv('datos_procesados.csv', index=False)
print(f"\n💾 Datos procesados guardados en 'datos_procesados.csv'")

print(f"\n" + "=" * 70)
print("🎉 PRÁCTICA COMPLETADA EXITOSAMENTE")
print("=" * 70)
print(f"📁 Archivos generados:")
print(f"   • metricas_supervisado.png")
print(f"   • clustering_resultados.png")
print(f"   • datos_procesados.csv")